In [ ]:
# Importar o drive para importar o arquivo das pastas
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import os
import csv
import re
import time
import spacy
nlp = spacy.load("en_core_web_sm")

FILE_KW = "/content/drive/My Drive/TCC 2025/Base_de_dados/saida_palavras_agrupadas_questoes_especificas.csv"
ARQUIVO_HISTORIAS = "/content/drive/My Drive/TCC 2025/Base_de_dados/historias_nao_utilizadas.txt"

def carregar_keywords_com_qes(caminho_arquivo):
    inicio = time.time()
    palavras_qes = {}
    if os.path.exists(caminho_arquivo):
        with open(caminho_arquivo, "r", encoding="utf-8") as arquivo_csv:
            leitor = csv.DictReader(arquivo_csv)
            for linha in leitor:
                palavra = linha['palavra'].strip()
                if palavra:
                    qes = [qe for qe, valor in linha.items() if qe.startswith("QE") and valor.strip() == '1']
                    palavras_qes[palavra] = qes
    else:
        print("Arquivo não encontrado.")
    fim = time.time()
    print(f"Tempo para carregar keywords: {fim - inicio:.4f} segundos")
    return palavras_qes

def extrair_historia_usuario(caminho_arquivo):
    inicio = time.time()
    historias_processadas = []
    with open(caminho_arquivo, "r", encoding="utf-8") as arquivo:
        for linha in arquivo:
            linha = linha.strip()
            if linha:
                if ", I want " in linha:
                    historia_usuario = linha.split(", I want ", 1)[-1]
                    historias_processadas.append(historia_usuario)
                elif "I want " in linha:
                    historia_usuario = linha.split("I want ", 1)[-1]
                    historias_processadas.append(historia_usuario)
    fim = time.time()
    print(f"Tempo para extrair histórias: {fim - inicio:.4f} segundos")
    return historias_processadas

def buscar_qes_em_historia(historia_usuario, palavras_qes):
    palavras_encontradas = []
    qes_total_BD = set()
    dic_palavra_qe_BD = {}

    # Transforma as palavras-chave em um set para busca rápida
    palavras_chave_set = set(palavras_qes.keys())

    # Tokeniza a história em palavras, mantendo apenas palavras simples
    palavras_historia = re.findall(r'\b\w+\b', historia_usuario.lower())

    for palavra in palavras_historia:
        if palavra in palavras_chave_set:
            if palavra not in dic_palavra_qe_BD:  # evita repetir
                palavras_encontradas.append(palavra)
                qes = palavras_qes.get(palavra, [])
                qes_total_BD.update(qes)
                dic_palavra_qe_BD[palavra] = qes

    return palavras_encontradas, dic_palavra_qe_BD, sorted(qes_total_BD)

def buscar_qes_em_historia_lemmatizada(historia_usuario, palavras_qes):
    # Mapear lemas das palavras-chave para as palavras originais
    palavras_chave_lem = {}
    for palavra in palavras_qes.keys():
        if palavra.strip() != "":
            doc = nlp(palavra.lower())
            if len(doc) > 0:
                palavras_chave_lem[doc[0].lemma_] = palavra

    # Lematizar a história do usuário
    doc_historia = nlp(historia_usuario.lower())

    palavras_encontradas = []
    dic_palavra_qe = {}
    qes_total = set()

    for token in doc_historia:
        # Verificação explícita para a palavra exata "data"
        if token.text.lower() == "data":
            if "data" not in dic_palavra_qe and "data" in palavras_qes:
                palavras_encontradas.append("data")
                qes = palavras_qes.get("data", [])
                qes_total.update(qes)
                dic_palavra_qe["data"] = qes

        # Verifica se o lema está no dicionário
        lema = token.lemma_
        if lema in palavras_chave_lem:
            palavra_original = palavras_chave_lem[lema]
            if palavra_original not in dic_palavra_qe:
                palavras_encontradas.append(palavra_original)
                qes = palavras_qes.get(palavra_original, [])
                qes_total.update(qes)
                dic_palavra_qe[palavra_original] = qes

    return palavras_encontradas, dic_palavra_qe, sorted(qes_total)




# Execução principal
palavras_qes = carregar_keywords_com_qes(FILE_KW)
palavras_chave = list(palavras_qes.keys())
historias_extraidas = extrair_historia_usuario(ARQUIVO_HISTORIAS)
#historias_extraidas = historias_extraidas[:1]

inicio = time.time()
for idx, historia_usuario in enumerate(historias_extraidas, start=1):
    print(f"\n{'='*60}")
    print(f"História {idx}: {historia_usuario}\n")
    print("Palavras-chave encontradas e suas QEs correspondentes:")

    palavras_encontradas, dic_palavra_qe_BD, qes_total = buscar_qes_em_historia(historia_usuario, palavras_qes)

    for palavra, qes in dic_palavra_qe_BD.items():
       print(f" - {palavra}: {qes}")

    print("\nConjunto total de QE's encontradas:")
    print(qes_total)

fim = time.time()
print(f"Tempo para busca direta: {fim - inicio:.4f} segundos")

# Arquivo de saída
ARQUIVO_SAIDA = "/content/drive/My Drive/TCC 2025/Base_de_dados/qes_por_historia_busca_direta.txt"
quant_busca_direta = []

inicio = time.time()
with open(ARQUIVO_SAIDA, "w", encoding="utf-8") as arquivo_saida:
    for idx, historia_usuario in enumerate(historias_extraidas, start=1):
        palavras_encontradas, dic_palavra_qe_BD, qes_total = buscar_qes_em_historia(historia_usuario, palavras_qes)

        # Converte o conjunto de QEs em string separada por vírgula
        if len(qes_total) != 0:
          linha_qes = ",".join(qes_total)
          quant_busca_direta.append(len(qes_total))
        else:
          linha_qes = " " #"QE1,QE10,QE11,QE12,QE13,QE14,QE15,QE16,QE17,QE18,QE2,QE3,QE4,QE5,QE6,QE7,QE8,QE9"
          quant_busca_direta.append(0)

        # Escreve no arquivo (uma linha por história)
        arquivo_saida.write(f"{linha_qes}\n")

fim = time.time()

economia_busca_direta = []
print(f"Tempo de processamento e gravação (busca direta): {fim - inicio:.4f} segundos")
#print(f"Resultados salvos em: {ARQUIVO_SAIDA}")
print(quant_busca_direta)
print(sum(quant_busca_direta))
soma = 18*len(historias_extraidas)-sum(quant_busca_direta)
print(f"Taxa de diminuição (busca direta): {1 - sum(quant_busca_direta)/(18*len(historias_extraidas))}")
#print(f"Soma: {soma}")




# Arquivo de saída para lematização
ARQUIVO_SAIDA_LEMA = "/content/drive/My Drive/TCC 2025/Base_de_dados/qes_por_historia_lemmatizada.txt"
quant_busca_lema = []

inicio = time.time()
with open(ARQUIVO_SAIDA_LEMA, "w", encoding="utf-8") as arquivo_saida:
    for idx, historia_usuario in enumerate(historias_extraidas, start=1):
        print(f"\n{'='*60}")
        print(f"História {idx}: {historia_usuario}\n")
        print("Palavras-chave (lematizadas) encontradas e suas QEs correspondentes:")

        palavras_encontradas, dic_palavra_qe_lema, qes_total_lema = buscar_qes_em_historia_lemmatizada(historia_usuario, palavras_qes)

        for palavra, qes in dic_palavra_qe_lema.items():
            print(f" - {palavra}: {qes}")

        print("\nConjunto total de QE's encontradas:")
        print(qes_total_lema)

        # Converte o conjunto de QEs em string separada por vírgula
        if len(qes_total_lema) != 0:
            linha_qes = ",".join(qes_total_lema)
            quant_busca_lema.append(len(qes_total_lema))
        else:
            linha_qes = " " #"QE1,QE10,QE11,QE12,QE13,QE14,QE15,QE16,QE17,QE18,QE2,QE3,QE4,QE5,QE6,QE7,QE8,QE9"
            quant_busca_lema.append(0)

        # Escreve no arquivo (uma linha por história)
        arquivo_saida.write(f"{linha_qes}\n")

fim = time.time()

print(f"Tempo de processamento e gravação (lemmatização): {fim - inicio:.4f} segundos")
print(quant_busca_lema)
print(sum(quant_busca_lema))
taxa_reducao_lema = 1 - sum(quant_busca_lema) / (18 * len(historias_extraidas))
print(f"Taxa de diminuição (lemmatização): {taxa_reducao_lema:.4f}")
dif = []
for i in range(len(quant_busca_direta)):
  dif.append(quant_busca_lema[i]-quant_busca_direta[i])
  if(dif[i]<0):
    print(i+1)
print(dif)

Tempo para carregar keywords: 0.0075 segundos
Tempo para extrair histórias: 0.0043 segundos

História 1: to be able to keep track of which camper submitted which forms, so that legal issues are avoided.

Palavras-chave encontradas e suas QEs correspondentes:
 - keep: ['QE7', 'QE8', 'QE16']
 - track: ['QE7', 'QE8', 'QE10', 'QE11', 'QE16', 'QE18']
 - camper: []

Conjunto total de QE's encontradas:
['QE10', 'QE11', 'QE16', 'QE18', 'QE7', 'QE8']

História 2: to Remove a Hold, so that I can allow progression through the workflow or other actions in the system now that the issue has been resolved.

Palavras-chave encontradas e suas QEs correspondentes:
 - hold: []
 - allow: ['QE7', 'QE8', 'QE10']
 - workflow: ['QE2', 'QE7', 'QE8', 'QE9', 'QE18']
 - other: []
 - system: ['QE2', 'QE5', 'QE6', 'QE7', 'QE8', 'QE9', 'QE10', 'QE11', 'QE16', 'QE18']
 - issue: []

Conjunto total de QE's encontradas:
['QE10', 'QE11', 'QE16', 'QE18', 'QE2', 'QE5', 'QE6', 'QE7', 'QE8', 'QE9']

História 3: to specify th

Tempo para carregar keywords: 0.0135 segundos
Tempo para extrair histórias: 0.0027 segundos
Tempo para busca direta: 0.0097 segundos
Tempo de processamento e gravação (busca direta): 0.0171 segundos
[3, 10, 3, 18, 0, 4, 11, 13, 18, 12, 14, 3, 8, 3, 13, 4, 0, 1, 3, 6, 0, 6, 4, 0, 10, 11, 5, 6, 2, 18, 2, 3, 4, 9, 12, 11, 0, 10, 7, 11, 6, 4, 6, 7, 9, 7, 4, 13, 14, 10, 6, 0, 10, 8, 10, 0, 7, 3, 10, 11, 11, 10, 2, 13, 10, 10, 8, 13, 11, 0, 4, 8, 3, 9, 12, 4, 10, 9, 6, 0, 8, 5, 4, 10, 5, 5, 11, 10, 7, 0, 3, 11, 2, 13, 10, 11, 4, 11, 5, 3, 3, 13, 10, 10, 8, 10, 10, 15, 2, 13, 4, 12, 10, 7, 11, 0, 3, 7, 6, 3, 0, 6, 5, 8, 5, 10, 10, 4, 2, 6, 11, 5, 12, 10, 10, 12, 2, 4, 5, 5, 7, 3, 3]
Taxa de diminuição (busca direta): 0.6052836052836053
Tempo de processamento e gravação (lemmatização): 1078.9268 segundos
[8, 10, 4, 18, 2, 8, 8, 15, 18, 12, 6, 3, 9, 3, 13, 7, 3, 1, 3, 6, 0, 8, 4, 3, 11, 11, 5, 6, 3, 18, 3, 3, 4, 9, 12, 12, 0, 10, 7, 11, 7, 4, 6, 6, 9, 7, 5, 13, 14, 10, 6, 0, 10, 8, 10, 2, 8, 3, 10, 11, 11, 10, 5, 13, 10, 10, 7, 14, 11, 2, 7, 8, 11, 10, 12, 4, 11, 9, 6, 0, 8, 5, 8, 10, 5, 5, 11, 10, 7, 6, 6, 11, 2, 13, 10, 11, 4, 11, 6, 4, 3, 13, 13, 10, 7, 10, 11, 15, 2, 13, 3, 10, 12, 7, 5, 3, 3, 7, 8, 5, 0, 2, 10, 8, 5, 0, 10, 4, 2, 6, 11, 5, 12, 8, 10, 12, 2, 4, 5, 6, 7, 5, 3]
Taxa de diminuição (lemmatização): 0.5855